In [1]:
import requests
import json
import pandas
from request_who import get_indicator_data, get_indicator_code
import numpy as np

In [2]:
INDICATORS_WHO = [
    "Life expectancy at birth (years)",
    "Age-standardized suicide rates (per 100 000 population)",
    "Primary data availability for UHC Service Coverage Index (SDG 3.8.1) (%)",
    "Prevalence of anaemia in women of reproductive age (aged 15-49) (%)",
    "Poliomyelitis - number of reported cases",
    "Estimates of rates of homicides per 100 000 population",
    "Prevalence of obesity among adults, BMI &GreaterEqual; 30 (age-standardized estimate) (%)"
]


In [3]:
dict_who = get_indicator_code(INDICATORS_WHO)
for k, v in dict_who.items():
    print(f"{k} : {v}")

MH_12 : Age-standardized suicide rates (per 100 000 population)
NCD_BMI_30A : Prevalence of obesity among adults, BMI &GreaterEqual; 30 (age-standardized estimate) (%)
UHC_AVAILABILITY_SCORE : Primary data availability for UHC Service Coverage Index (SDG 3.8.1) (%)
VIOLENCE_HOMICIDERATE : Estimates of rates of homicides per 100 000 population
WHOSIS_000001 : Life expectancy at birth (years)
WHS3_49 : Poliomyelitis - number of reported cases
NUTRITION_ANAEMIA_REPRODUCTIVEAGE_PREV : Prevalence of anaemia in women of reproductive age (aged 15-49) (%)


In [4]:
pd_indicators = []

NCD_BMI_30A = get_indicator_data("NCD_BMI_30A", "Dim1 eq 'BTSX' and TimeDim eq 2015 and SpatialDimType eq 'COUNTRY'")
pd_indicators.append(NCD_BMI_30A)

WHOSIS_000001 = get_indicator_data("WHOSIS_000001", "Dim1 eq 'BTSX' and TimeDim eq 2015 and SpatialDimType eq 'COUNTRY'")
pd_indicators.append(WHOSIS_000001)

MH_12 = get_indicator_data("MH_12", "Dim1 eq 'BTSX' and TimeDim eq 2015 and SpatialDimType eq 'COUNTRY'")
pd_indicators.append(MH_12)

In [5]:
UHC_AVAILABILITY_SCORE = get_indicator_data("UHC_AVAILABILITY_SCORE", "SpatialDimType eq 'COUNTRY'")
pd_indicators.append(UHC_AVAILABILITY_SCORE)

VIOLENCE_HOMICIDERATE = get_indicator_data("VIOLENCE_HOMICIDERATE", "Dim1 eq 'BTSX' and TimeDim eq 2015")
pd_indicators.append(VIOLENCE_HOMICIDERATE)

In [6]:
WHS3_49 = get_indicator_data("WHS3_49", "TimeDim eq 2015")
pd_indicators.append(WHS3_49)

NUTRITION_ANAEMIA_REPRODUCTIVEAGE_PREV = get_indicator_data("NUTRITION_ANAEMIA_REPRODUCTIVEAGE_PREV", "TimeDim eq 2015 and Dim1 eq 'TOTAL' and SpatialDimType eq 'COUNTRY'")
pd_indicators.append(NUTRITION_ANAEMIA_REPRODUCTIVEAGE_PREV)

In [7]:
full_data_table = pandas.merge(NCD_BMI_30A, WHOSIS_000001, how="left", on="CountryCode")
for i in pd_indicators[2:]:
    full_data_table = pandas.merge(full_data_table, i, how="left", on="CountryCode")

print(full_data_table)

    CountryCode  NCD_BMI_30A  WHOSIS_000001    MH_12  UHC_AVAILABILITY_SCORE  \
0           AFG          5.2       61.65429   5.9900                88.88889   
1           ALB         21.1       77.84428   4.2300                79.86111   
2           DZA         26.7       76.54363   2.7200                85.41667   
3           AND         25.4            NaN      NaN                31.25000   
4           AGO          7.9       61.71907  13.2696                69.44444   
..          ...          ...            ...      ...                     ...   
190         ZWE         15.2       58.47603  30.7352                75.00000   
191         MCO          NaN            NaN      NaN                34.02778   
192         SMR          NaN            NaN      NaN                25.69444   
193         SSD          NaN       61.98944   6.4100                48.61111   
194         SDN          NaN       67.93808   4.8900                57.63889   

     VIOLENCE_HOMICIDERATE  WHS3_49  NU

In [8]:
full_data_table['had_polio'] = np.where(full_data_table['WHS3_49']>0, 1, 0)

full_data_table = full_data_table[['CountryCode', 'WHOSIS_000001', 'NCD_BMI_30A', 'MH_12',
                               'UHC_AVAILABILITY_SCORE', 'VIOLENCE_HOMICIDERATE', 'WHS3_49',
                               'NUTRITION_ANAEMIA_REPRODUCTIVEAGE_PREV', 'had_polio']]

print(full_data_table)

    CountryCode  WHOSIS_000001  NCD_BMI_30A    MH_12  UHC_AVAILABILITY_SCORE  \
0           AFG       61.65429          5.2   5.9900                88.88889   
1           ALB       77.84428         21.1   4.2300                79.86111   
2           DZA       76.54363         26.7   2.7200                85.41667   
3           AND            NaN         25.4      NaN                31.25000   
4           AGO       61.71907          7.9  13.2696                69.44444   
..          ...            ...          ...      ...                     ...   
190         ZWE       58.47603         15.2  30.7352                75.00000   
191         MCO            NaN          NaN      NaN                34.02778   
192         SMR            NaN          NaN      NaN                25.69444   
193         SSD       61.98944          NaN   6.4100                48.61111   
194         SDN       67.93808          NaN   4.8900                57.63889   

     VIOLENCE_HOMICIDERATE  WHS3_49  NU

In [9]:
full_data_table.to_csv(path_or_buf="./datacsv/who_data.csv", index=False)